# Prerequisite and libraries

In [1]:
MAX_LEN = 192 
DROPOUT = 0.3 # use aggressive dropout
BATCH_SIZE = 16 # per TPU core
TOTAL_STEPS_STAGE1 = 2000
VALIDATE_EVERY_STAGE1 = 200
TOTAL_STEPS_STAGE2 = 200
VALIDATE_EVERY_STAGE2 = 10

### Different learning rate for transformer and head ###
LR_TRANSFORMER = 5e-6
LR_HEAD = 1e-3

PRETRAINED_TOKENIZER=  'jplu/tf-xlm-roberta-large'
PRETRAINED_MODEL = '/kaggle/input/jigsaw-mlm-finetuned-xlm-r-large'
D = '/kaggle/input/jigsaw-multilingual-toxic-comment-classification/'
D_TRANS = '/kaggle/input/jigsaw-train-multilingual-coments-google-api/'
D_BIAS = '../input/translated-train-bias-all-langs/All languages/'
import gc
import os
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import transformers
from transformers import TFRobertaModel, AutoTokenizer
import logging
# no extensive logging 
logging.getLogger().setLevel(logging.NOTSET)

AUTO = tf.data.experimental.AUTOTUNE

2.2.0


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


## Connect to TPU

In [2]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


 ## Load text data into memory
 
 - Traning data is englih + all translations. The reason to use english too is that people use english in their foreign language comments all the time.
 - Not using the full dataset, downsampling negatives to 50-50%

In [3]:
def load_jigsaw_trans(langs=['tr','it','es','ru','fr','pt'], 
                      columns=['comment_text', 'toxic']):
    train_6langs=[]
    for i in range(len(langs)):
        fn = D_TRANS+'jigsaw-toxic-comment-train-google-%s-cleaned.csv'%langs[i]
        train_6langs.append(downsample(pd.read_csv(fn)[columns]))

    return train_6langs

def downsample(df):
    """Subsample the train dataframe to 50%-50%"""
    ds_df= pd.concat([
        df.query('toxic==1'),
        df.query('toxic==0').sample(sum(df.toxic))
    ])
    
    return ds_df
    

train_df = pd.concat(load_jigsaw_trans()) 
val_df = pd.read_csv(D+'validation.csv')
test_df = pd.read_csv(D+'test.csv')
sub_df = pd.read_csv(D+'sample_submission.csv')

In [4]:
len(train_df)

256496

In [5]:
import os
import pandas as pd
external = '/kaggle/input/toxic-comment-detection-multilingual-extended/archive/'
e_ru = pd.read_csv(f'{external}russian/labeled.csv')

e_tr = pd.read_csv(f'{external}turkish/troff-v1.0.tsv', sep='\t', header=0)
e_tr.label = e_tr.label.apply(lambda x: 1 if x not in ['non', 'prof'] else 0)

e_it = pd.concat([
    pd.read_csv(f'{external}italian/haspeede_FB-train.tsv', sep='\t', header=0),
    pd.read_csv(f'{external}italian/haspeede_TW-train.tsv', sep='\t', header=0)
])
e_ru.rename(columns={'comment':'comment_text'}, inplace=True)
e_tr.rename(columns={'text':'comment_text', 'label':'toxic'}, inplace=True)
e_it.rename(columns={'comment':'comment_text'}, inplace=True)

In [6]:
trans_bias=pd.DataFrame()
for file in os.listdir(D_BIAS):
    if file.startswith("train-bias-toxic-google-api") and file.endswith("cleaned.csv"):
        df = pd.read_csv(D_BIAS+file,usecols=['comment_text','toxic'])
        df.toxic = df.toxic.round().astype(int)
        toxic = df[['comment_text', 'toxic']].query('toxic==1').sample(20000,random_state=42)
        in_toxic = df[['comment_text', 'toxic']].query('toxic==0').sample(20000,random_state=42)
        trans_bias=trans_bias.append(pd.concat([in_toxic,toxic]))
#In train
print("Train Toxic trans_bias = ",len(trans_bias[['comment_text', 'toxic']].query('toxic==1')))
print("Train Intoxic trans_bias = ",len(trans_bias[['comment_text', 'toxic']].query('toxic==0')))
trans_bias = trans_bias.drop_duplicates(subset='comment_text')
trans_bias = trans_bias.reset_index(drop=True)

Train Toxic trans_bias =  120000
Train Intoxic trans_bias =  120000


In [7]:
train_df = pd.concat([train_df,trans_bias,e_ru,e_tr,e_it],axis = 0).reset_index(drop=True)
train_df = train_df.drop_duplicates(subset='comment_text')
train_df = train_df.reset_index(drop=True)
train_df = train_df.sample(frac=1).reset_index(drop=True)
del e_ru,e_tr,e_it,trans_bias
gc.collect()

69

## Tokenize  it with the models own tokenizer

- Note it takes some time ( approx 5 minutes)
- Note, we need to reshape the targets

In [8]:
%%time

def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])
    

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_TOKENIZER)
X_train = regular_encode(train_df.comment_text.values, tokenizer, maxlen=MAX_LEN)
X_val = regular_encode(val_df.comment_text.values, tokenizer, maxlen=MAX_LEN)
X_test = regular_encode(test_df.content.values, tokenizer, maxlen=MAX_LEN)

y_train = train_df.toxic.values.reshape(-1,1)
y_val = val_df.toxic.values.reshape(-1,1)

CPU times: user 8min 31s, sys: 2.54 s, total: 8min 34s
Wall time: 8min 35s


## Create distributed tensorflow datasets

- Note, validation dataset does not contain labels, we keep track of it ourselves

In [9]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))
        
    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset
    
    
train_dist_dataset = create_dist_dataset(X_train, y_train, True)
val_dist_dataset   = create_dist_dataset(X_val)
test_dist_dataset  = create_dist_dataset(X_test)

## Build model from pretrained transformer
The reasoning is the following, the transformer is trained for super long time and has a very good multilingual representaton, which we only want to change a little, while the head needs to be trained from scratch.

We define 2 separate optimizers for the transofmer and the head layer. This is a simple way to use different learning rate for the transformer and the head. The caffe style "lr_multiplier" option would be more elegant but that is not available in keras.

We add the name 'custom' to the head layer, so that we can find it later and use a different learning rate with this layer

In [10]:
%%time
from tensorflow.keras.layers import  Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D,Average,Dropout,concatenate
def create_model_and_optimizer():
    with strategy.scope():
        transformer_layer = TFRobertaModel.from_pretrained(PRETRAINED_MODEL)                
        model = build_model1(transformer_layer)
        optimizer_transformer = Adam(learning_rate=LR_TRANSFORMER)
        optimizer_head = Adam(learning_rate=LR_HEAD)
    return model, optimizer_transformer, optimizer_head


def build_model1(transformer):
    inp = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    # Huggingface transformers have multiple outputs, embeddings are the first one
    # let's slice out the first position, the paper says its not worse than pooling
    x = transformer(inp)[0][:, 0, :]  
    x = Dropout(DROPOUT)(x)
    ### note, adding the name to later identify these weights for different LR
    out = Dense(1, activation='sigmoid', name='custom_head')(x)
    model = Model(inputs=[inp], outputs=[out])
    
    return model
def build_model2(transformer):
    input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    apool= GlobalAveragePooling1D()(sequence_output)
    mpol = GlobalMaxPooling1D()(sequence_output)
    cat = concatenate([
        apool,
        mpol
    ])
    x = Dropout(DROPOUT)(cat)
    out = Dense(1, activation='sigmoid',name='custom_head')(x)
    model = Model(inputs=[input_word_ids], outputs=[out])

    return model

def build_model3(transformer):
    input_word_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")
    seq_out = transformer(input_word_ids)[0]
    pool= GlobalAveragePooling1D()(seq_out)
    
    dense=[]
    FC = Dense(32,activation='relu')
    for p in np.linspace(0.2,0.5,3):
        x=Dropout(p)(pool)
        x=FC(x)
        x=Dense(1,activation='sigmoid')(x)
        dense.append(x)
    
    out = Average()(dense)
    model = Model(inputs=[input_word_ids], outputs=[out])
    
    return model


model, optimizer_transformer, optimizer_head = create_model_and_optimizer()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 192, 1024), (None 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 1024)              0         
_________________________________________________________________
custom_head (Dense)          (None, 1)                 1025      
Total params: 559,891,457
Trainable params: 559,891,457
Non-trainable params: 0
_________________________________________________________________
CPU times: user 38.4 s, sys: 34 s, total: 1min 12s
Wall time: 1min 9s


### Define stuff for the custom training loop

We will need:
- 1, losses, and  optionally a training AUC metric here: these need to be defined in the scope of the distributed strategy. 
- 2, A full training loop
- 3, A distributed train step called in the training loop, which uses a single replica train step
- 4, A prediction loop with dstibute 


At the end of training we restore the parameters which had the best validation score.


For the different learning rate we need to apply gradients in two steps, check the train_step function for details.



- Note, we are using exact AUC, for the valdationdata, and approximate AUC for the training data

In [11]:
def define_losses_and_metrics():
    with strategy.scope():
        loss_object = tf.keras.losses.BinaryCrossentropy(
            reduction=tf.keras.losses.Reduction.NONE, from_logits=False)

        def compute_loss(labels, predictions):
            per_example_loss = loss_object(labels, predictions)
            loss = tf.nn.compute_average_loss(
                per_example_loss, global_batch_size = global_batch_size)
            return loss

        train_accuracy_metric = tf.keras.metrics.AUC(name='training_AUC')

    return compute_loss, train_accuracy_metric


def train(train_dist_dataset, val_dist_dataset=None, y_val=None,
          total_steps=2000, validate_every=200):
    best_weights, history = None, []
    step = 0
    ### Training lopp ###
    for tensor in train_dist_dataset:
        distributed_train_step(tensor) 
        step+=1

        if (step % validate_every == 0):   
            ### Print train metrics ###  
            train_metric = train_accuracy_metric.result().numpy()
            print("Step %d, train AUC: %.5f" % (step, train_metric))   
            
            ### Test loop with exact AUC ###
            if val_dist_dataset:
                val_metric = roc_auc_score(y_val, predict(val_dist_dataset))
                print("Step %d,   val AUC: %.5f" %  (step,val_metric))   
                
                # save weights if it is the best yet
                history.append(val_metric)
                if history[-1] == max(history):
                    best_weights = model.get_weights()

            ### Reset (train) metrics ###
            train_accuracy_metric.reset_states()
            
        if step  == total_steps:
            break
    
    ### Restore best weighths ###
    model.set_weights(best_weights)



@tf.function
def distributed_train_step(data):
    strategy.experimental_run_v2(train_step, args=(data,))

def train_step(inputs):
    features, labels = inputs
    
    ### get transformer and head separate vars
    # get rid of pooler head with None gradients
    transformer_trainable_variables = [ v for v in model.trainable_variables 
                                       if (('pooler' not in v.name)  and 
                                           ('custom' not in v.name))]
    head_trainable_variables = [ v for v in model.trainable_variables 
                                if 'custom'  in v.name]

    # calculate the 2 gradients ( note persistent, and del)
    with tf.GradientTape(persistent=True) as tape:
        predictions = model(features, training=True)
        loss = compute_loss(labels, predictions)
    gradients_transformer = tape.gradient(loss, transformer_trainable_variables)
    gradients_head = tape.gradient(loss, head_trainable_variables)
    del tape
        
    ### make the 2 gradients steps
    optimizer_transformer.apply_gradients(zip(gradients_transformer, 
                                              transformer_trainable_variables))
    optimizer_head.apply_gradients(zip(gradients_head, 
                                       head_trainable_variables))

    train_accuracy_metric.update_state(labels, predictions)



def predict(dataset):  
    predictions = []
    for tensor in dataset:
        predictions.append(distributed_prediction_step(tensor))
    ### stack replicas and batches
    predictions = np.vstack(list(map(np.vstack,predictions)))
    return predictions

@tf.function
def distributed_prediction_step(data):
    predictions = strategy.experimental_run_v2(prediction_step, args=(data,))
    return strategy.experimental_local_results(predictions)

def prediction_step(inputs):
    features = inputs  # note datasets used in prediction do not have labels
    predictions = model(features, training=False)
    return predictions


compute_loss, train_accuracy_metric = define_losses_and_metrics()

## Finally train it on english comments


- Note it takes some time
- Don't mind the warning: "Converting sparse IndexedSlices to a dense Tensor"

In [12]:
%%time
train(train_dist_dataset, val_dist_dataset, y_val,
      TOTAL_STEPS_STAGE1, VALIDATE_EVERY_STAGE1)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


Step 200, train AUC: 0.82907
Step 200,   val AUC: 0.92894
Step 400, train AUC: 0.93743
Step 400,   val AUC: 0.93307
Step 600, train AUC: 0.94271
Step 600,   val AUC: 0.93755
Step 800, train AUC: 0.94864
Step 800,   val AUC: 0.93314
Step 1000, train AUC: 0.95307
Step 1000,   val AUC: 0.93767
Step 1200, train AUC: 0.95232
Step 1200,   val AUC: 0.93776
Step 1400, train AUC: 0.95624
Step 1400,   val AUC: 0.93518
Step 1600, train AUC: 0.95618
Step 1600,   val AUC: 0.93577
Step 1800, train AUC: 0.95830
Step 1800,   val AUC: 0.94019
Step 2000, train AUC: 0.96155
Step 2000,   val AUC: 0.93649
CPU times: user 4min 16s, sys: 1min 4s, total: 5min 20s
Wall time: 18min 38s


## Finetune it on the validation data

In [13]:
%%time

# decrease LR for second stage in the head
optimizer_head.learning_rate.assign(1e-4)

# split validation data into train test
X_train, X_val, y_train, y_val = train_test_split(X_val, y_val, test_size = 0.1)

# make a datasets
train_dist_dataset = create_dist_dataset(X_train, y_train, training=True)
val_dist_dataset = create_dist_dataset(X_val, y_val)

# train again
train(train_dist_dataset, val_dist_dataset, y_val,
      total_steps = TOTAL_STEPS_STAGE2, 
      validate_every = VALIDATE_EVERY_STAGE2)  # not validating but printing now

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 256002048 elements. This may consume a large amount of memory.
  num_elements)


Step 10, train AUC: 0.92700
Step 10,   val AUC: 0.96192
Step 20, train AUC: 0.93180
Step 20,   val AUC: 0.96225
Step 30, train AUC: 0.93459
Step 30,   val AUC: 0.96387
Step 40, train AUC: 0.94977
Step 40,   val AUC: 0.96437
Step 50, train AUC: 0.94555
Step 50,   val AUC: 0.96384
Step 60, train AUC: 0.93536
Step 60,   val AUC: 0.96289
Step 70, train AUC: 0.95708
Step 70,   val AUC: 0.96945
Step 80, train AUC: 0.95547
Step 80,   val AUC: 0.96506
Step 90, train AUC: 0.94959
Step 90,   val AUC: 0.95925
Step 100, train AUC: 0.96000
Step 100,   val AUC: 0.96639
Step 110, train AUC: 0.95889
Step 110,   val AUC: 0.96753
Step 120, train AUC: 0.96691
Step 120,   val AUC: 0.96820
Step 130, train AUC: 0.97480
Step 130,   val AUC: 0.97086
Step 140, train AUC: 0.96716
Step 140,   val AUC: 0.97098
Step 150, train AUC: 0.95388
Step 150,   val AUC: 0.96701
Step 160, train AUC: 0.96734
Step 160,   val AUC: 0.96219
Step 170, train AUC: 0.96974
Step 170,   val AUC: 0.96270
Step 180, train AUC: 0.98154
Ste

## Make predictions and submission

In [14]:
%%time
sub_df['toxic'] = predict(test_dist_dataset)[:,0]
sub_df.to_csv('submission.csv', index=False)

CPU times: user 26 s, sys: 4.68 s, total: 30.7 s
Wall time: 1min 30s
